In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler
from keras.constraints import maxnorm
from keras.regularizers import l2
from random import shuffle

from matplotlib import pyplot as plt
%matplotlib inline

import cv2
import glob
import os
import re

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
train_path = "/home/aiml_test_user/Shaheen/trainDC"

ROWS = 128
COLS = 128
CHANNELS = 3

images = [img for img in os.listdir(train_path)]
images_dog = [img for img in os.listdir(train_path) if "dog" in img]
images_cat = [img for img in os.listdir(train_path) if "cat" in img]


train_list = images_dog + images_cat

shuffle(train_list)

train = np.ndarray(shape=(len(train_list),ROWS, COLS))
labels = np.ndarray(len(train_list))

for i, img_path in enumerate(train_list):
    img = cv2.imread(os.path.join(train_path, img_path), 0)
    img = cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    
    train[i] = img
    if "dog" in img_path:
        labels[i] = 1
    else:
        labels[i] = 0

In [4]:
train1 = np.array(train).reshape((-1, 1, 128, 128)).astype('float32')

train1.shape

## dividing by 255
train1 /= 255

In [5]:

## splitting the data into train and test
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train1, labels, test_size=0.3)


In [12]:
img_size = 128

## CNN

def cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(1, img_size, img_size), activation='relu'))
    model.add(Convolution2D(32, 3, 3, activation='relu',border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    return model


model1 = cnn_model()


model1.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [13]:
## fitting the model
batch_size = 32
nb_epoch = 20

model1.fit(x_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(x_test, y_test))

Train on 17500 samples, validate on 7500 samples
Epoch 1/20
17500/17500 [==============================] - 56s - loss: 0.6859 - acc: 0.5715 - val_loss: 0.6333 - val_acc: 0.6367
Epoch 2/20
17500/17500 [==============================] - 60s - loss: 0.6018 - acc: 0.6749 - val_loss: 0.5542 - val_acc: 0.7257
Epoch 3/20
17500/17500 [==============================] - 60s - loss: 0.5307 - acc: 0.7383 - val_loss: 0.4769 - val_acc: 0.7688
Epoch 4/20
17500/17500 [==============================] - 60s - loss: 0.4524 - acc: 0.7929 - val_loss: 0.4363 - val_acc: 0.7972
Epoch 5/20
17500/17500 [==============================] - 60s - loss: 0.3970 - acc: 0.8267 - val_loss: 0.3494 - val_acc: 0.8477
Epoch 6/20
17500/17500 [==============================] - 60s - loss: 0.3638 - acc: 0.8429 - val_loss: 0.3431 - val_acc: 0.8461
Epoch 7/20
17500/17500 [==============================] - 60s - loss: 0.3288 - acc: 0.8607 - val_loss: 0.3008 - val_acc: 0.8723
Epoch 8/20
17500/17500 [==============================]

In [14]:
validation = model1.evaluate(x_test, y_test, verbose=1)
print('Test accuracy:', validation[1])

7488/7500 [============================>.] - ETA: 0sTest accuracy: 0.890266666667


In [8]:
img_size = 128

def catdog():
    
    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(1, ROWS, COLS), activation='relu'))
    model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model


model3 = catdog()


model3.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [9]:
## fitting the model
batch_size = 32
nb_epoch = 14

model3.fit(x_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(x_test, y_test))

Train on 17500 samples, validate on 7500 samples
Epoch 1/14
17500/17500 [==============================] - 68s - loss: 0.7112 - acc: 0.5230 - val_loss: 0.6821 - val_acc: 0.5665
Epoch 2/14
17500/17500 [==============================] - 71s - loss: 0.6714 - acc: 0.6006 - val_loss: 0.6412 - val_acc: 0.6297
Epoch 3/14
17500/17500 [==============================] - 72s - loss: 0.5885 - acc: 0.6991 - val_loss: 0.7378 - val_acc: 0.6780
Epoch 4/14
17500/17500 [==============================] - 72s - loss: 0.5102 - acc: 0.7561 - val_loss: 0.4360 - val_acc: 0.8001
Epoch 5/14
17500/17500 [==============================] - 72s - loss: 0.4598 - acc: 0.7907 - val_loss: 0.3981 - val_acc: 0.8225
Epoch 6/14
17500/17500 [==============================] - 72s - loss: 0.4071 - acc: 0.8191 - val_loss: 0.3461 - val_acc: 0.8439
Epoch 7/14
17500/17500 [==============================] - 72s - loss: 0.3687 - acc: 0.8441 - val_loss: 0.3041 - val_acc: 0.8637
Epoch 8/14
17500/17500 [==============================]

In [15]:
validation = model3.evaluate(x_test, y_test, verbose=1)
print('Test accuracy:', validation[1])

7500/7500 [==============================] - 5s     
Test accuracy: 0.902133333365


In [16]:
## data augmentation
X_train, X_val, Y_train, Y_val = train_test_split(train1, labels, test_size=0.2)


datagen = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)

datagen1 = ImageDataGenerator(rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True)


datagen1.fit(X_train)

In [19]:
# reinitialise the model

model2 = cnn_model()


model2.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

nb_epoch = 15
batch_size = 32
model2.fit_generator(datagen1.flow(X_train, Y_train, batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, Y_val))

Epoch 1/15
20000/20000 [==============================] - 64s - loss: 0.7020 - acc: 0.5367 - val_loss: 0.6848 - val_acc: 0.5396
Epoch 2/15
20000/20000 [==============================] - 67s - loss: 0.6532 - acc: 0.6191 - val_loss: 0.6150 - val_acc: 0.6528
Epoch 3/15
20000/20000 [==============================] - 67s - loss: 0.6075 - acc: 0.6736 - val_loss: 0.5276 - val_acc: 0.7264
Epoch 4/15
20000/20000 [==============================] - 66s - loss: 0.5822 - acc: 0.6963 - val_loss: 0.5913 - val_acc: 0.6902
Epoch 5/15
20000/20000 [==============================] - 66s - loss: 0.5557 - acc: 0.7192 - val_loss: 0.4977 - val_acc: 0.7552
Epoch 6/15
20000/20000 [==============================] - 66s - loss: 0.5442 - acc: 0.7305 - val_loss: 0.4746 - val_acc: 0.7644
Epoch 7/15
20000/20000 [==============================] - 66s - loss: 0.5296 - acc: 0.7441 - val_loss: 0.4872 - val_acc: 0.7610
Epoch 8/15
20000/20000 [==============================] - 66s - loss: 0.5150 - acc: 0.7533 - val_loss: 0

In [20]:
validation = model2.evaluate(x_test, y_test, verbose=1)
print('Test accuracy:', validation[1])

7488/7500 [============================>.] - ETA: 0sTest accuracy: 0.860933333365


In [21]:
## uploading and processing test data
test_path = "/home/aiml_test_user/Shaheen/testDC"

ROWS = 128
COLS = 128
CHANNELS = 3

images = [img for img in os.listdir(test_path)]

files = [ os.path.join('/home/aiml_test_user/Shaheen/testDC',str(i)+'.jpg') for i in range(1,12501) ]


test = np.ndarray(shape=(len(files),ROWS, COLS))

for i, img_path in enumerate(images):
    img = cv2.imread(os.path.join(test_path, img_path), 0)
    img = cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    
    test[i] = img
    
    
#for fname in files:
    #img = cv2.imread(os.path.join(test_path, img_path), 0)
    #img = cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    
    #test[i] = img
                     
                     

In [22]:
test1 = np.array(test).reshape((-1, 1, 128, 128)).astype('float32')


## dividing by 255
test1 /= 255

test1.shape

(12500, 1, 128, 128)

In [23]:
## predicting the test data
y_pred = model3.predict(test1)

# save results
np.savetxt('submission_DogsvCatsKaggleProb.csv', np.c_[range(1,len(test1)+1),y_pred], delimiter=',', header = 'id,label', comments = '')